In [1]:
import pickle 
import torch 
import os 
import sys 
sys.path.append('../../../submodule') 
import torch_util 
from torch_util.imports import * 

with open(os.path.expanduser('~/dataset/OAG/OAG-CS-Venue/hg_full.dict.pkl'), 'rb') as fp: 
    graph_info = pickle.load(fp)
    
edge_index_dict = graph_info['edge_index_dict']
num_nodes_dict = graph_info['num_nodes_dict']

In [2]:
{ k: v.shape for k, v in edge_index_dict.items() }

{('venue', 'VP_Conference', 'paper'): torch.Size([2, 296775]),
 ('venue', 'VP_Journal', 'paper'): torch.Size([2, 228062]),
 ('venue', 'VP_Repository', 'paper'): torch.Size([2, 19216]),
 ('venue', 'VP_Patent', 'paper'): torch.Size([2, 191]),
 ('paper', 'PV_Conference', 'venue'): torch.Size([2, 296775]),
 ('paper', 'PV_Journal', 'venue'): torch.Size([2, 228062]),
 ('paper', 'PV_Repository', 'venue'): torch.Size([2, 19216]),
 ('paper', 'PV_Patent', 'venue'): torch.Size([2, 191]),
 ('paper', 'PF_L0', 'field'): torch.Size([2, 544371]),
 ('paper', 'PF_L3', 'field'): torch.Size([2, 866423]),
 ('paper', 'PF_L1', 'field'): torch.Size([2, 1197205]),
 ('paper', 'PF_L2', 'field'): torch.Size([2, 2337525]),
 ('paper', 'PF_L5', 'field'): torch.Size([2, 202221]),
 ('paper', 'PF_L4', 'field'): torch.Size([2, 303541]),
 ('paper', 'PA_last', 'author'): torch.Size([2, 429392]),
 ('paper', 'PA_other', 'author'): torch.Size([2, 662167]),
 ('paper', 'PA_first', 'author'): torch.Size([2, 454913]),
 ('field',

In [3]:
num_nodes_dict 

{'author': 510189,
 'field': 45717,
 'institution': 9079,
 'paper': 544244,
 'venue': 6934}

In [4]:
os.makedirs('./TransE_emb', exist_ok=True)

def convert_hg_to_triplets(edge_index_dict: dict[EdgeType, EdgeIndex], 
                           num_nodes_dict: dict[NodeType, int],
                           output_path: str): 
    nid_offset_dict: dict[NodeType, int] = dict() 

    nid_offset = 0 

    for ntype, num_nodes in sorted(num_nodes_dict.items()):
        nid_offset_dict[ntype] = nid_offset 
        nid_offset += num_nodes 

    with open(output_path, 'w', encoding='utf-8') as fp:
        for etype, edge_index in tqdm(edge_index_dict.items()): 
            src_ntype, _, dest_ntype = etype 
            etype = '__'.join(etype) 
            src_nids, dest_nids = edge_index 
            
            src_nids.add_(nid_offset_dict[src_ntype])
            dest_nids.add_(nid_offset_dict[dest_ntype])

            for src_nid, dest_nid in zip(src_nids.tolist(), dest_nids.tolist()): 
                print(f"{src_nid}\t{etype}\t{dest_nid}", file=fp)

convert_hg_to_triplets(
    edge_index_dict = edge_index_dict, 
    num_nodes_dict = num_nodes_dict,
    output_path = './TransE_emb/triplets.tsv', 
)

  0%|          | 0/30 [00:00<?, ?it/s]